In [29]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

# Load the datasets
movies_df = pd.read_csv('/Users/rameshchidurala/Documents/DSC-530 Assignments/movies.csv')
ratings_df = pd.read_csv('/Users/rameshchidurala/Documents/DSC-530 Assignments/ratings.csv')

# Display the first few rows of each dataset for exploration
movies_df.head()




,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [30]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.000,964982703
1,1,3,4.000,964981247
2,1,6,4.000,964982224
3,1,47,5.000,964983815
4,1,50,5.000,964982931


In [31]:
df = pd.merge(ratings_df,movies_df,on='movieId')
df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.000,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.000,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.500,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.500,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.500,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [32]:
combine_movie_rating = df.dropna(axis = 0, subset = ['title'])
movie_ratingCount = (combine_movie_rating.
     groupby(by = ['title'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'totalRatingCount'})
     [['title', 'totalRatingCount']]
    )
movie_ratingCount.head()


,title,totalRatingCount
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2


In [33]:
rating_with_totalRatingCount = combine_movie_rating.merge(movie_ratingCount, left_on = 'title', right_on = 'title', how = 'left')
rating_with_totalRatingCount.head()

,userId,movieId,rating,timestamp,title,genres,totalRatingCount
0,1,1,4.000,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
1,5,1,4.000,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
2,7,1,4.500,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
3,15,1,2.500,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
4,17,1,4.500,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215


In [34]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(movie_ratingCount['totalRatingCount'].describe())

count   9719.000
mean      10.375
std       22.406
min        1.000
25%        1.000
50%        3.000
75%        9.000
max      329.000
Name: totalRatingCount, dtype: float64


In [35]:
movie_ratingCount.describe()

,totalRatingCount
count,9719.000
mean,10.375
std,22.406
min,1.000
25%,1.000
50%,3.000
75%,9.000
max,329.000


In [36]:
popularity_threshold = 50
rating_popular_movie= rating_with_totalRatingCount.query('totalRatingCount >= @popularity_threshold')
rating_popular_movie.head()

,userId,movieId,rating,timestamp,title,genres,totalRatingCount
0,1,1,4.000,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
1,5,1,4.000,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
2,7,1,4.500,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
3,15,1,2.500,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215
4,17,1,4.500,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215


In [37]:
rating_popular_movie.shape

(41362, 7)

In [38]:
# First lets create a Pivot matrix
movie_features_df=rating_popular_movie.pivot_table(index='title',columns='userId',values='rating').fillna(0)
movie_features_df.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You (1999),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,3.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000
12 Angry Men (1957),0.000,0.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000,0.000,...,5.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2001: A Space Odyssey (1968),0.000,0.000,0.000,0.000,0.000,0.000,4.000,0.000,0.000,0.000,...,0.000,0.000,5.000,0.000,0.000,5.000,0.000,3.000,0.000,4.500
28 Days Later (2002),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.500,0.000,5.000
300 (2007),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.000,...,0.000,0.000,0.000,0.000,3.000,0.000,0.000,5.000,0.000,4.000


In [39]:
from scipy.sparse import csr_matrix

movie_features_df_matrix = csr_matrix(movie_features_df.values)

from sklearn.neighbors import NearestNeighbors


model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(movie_features_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [40]:
import pandas as pd

def make_recommendations(model_knn, movie_features_df, query_movie_title, n_recommendations=5):
    """
    Generates movie recommendations similar to the query_movie_title.
    
    Parameters:
    - model_knn: Trained NearestNeighbors model.
    - movie_features_df: DataFrame containing movie features.
    - query_movie_title: Title of the movie to find recommendations for.
    - n_recommendations: Number of recommendations to return.
    
    Returns:
    - DataFrame containing recommended movies and their distances from the query movie.
    """
    movie_list = movie_features_df.index.tolist()
    
    # Check if the movie is in our database
    if query_movie_title not in movie_list:
        return "Movie not found. Please check the spelling or try another movie."
    
    # Convert the string movie title into a numeric index for the model
    query_index = movie_list.index(query_movie_title)
    
    # Reshape the movie features dataframe matrix and use the kNN model to find similar movies
    distances, indices = model_knn.kneighbors(movie_features_df.iloc[query_index,:].values.reshape(1, -1), n_neighbors=n_recommendations+1)
    
    # Initialize the list to store recommendations
    recommendations = []
    
    # Loop through the found indices, skipping the first one as it is the query movie itself
    for i in range(1, len(distances.flatten())):
        movie_title = movie_features_df.index[indices.flatten()[i]]
        movie_distance = distances.flatten()[i]
        recommendations.append((movie_title, movie_distance))
        
    return pd.DataFrame(recommendations, columns=['Movie Title', 'Distance'])


In [41]:
# Example usage to get recommendations for "Toy Story (1995)"
query_movie_title = 'Toy Story (1995)'
n_recommendations = 5

recommended_movies = make_recommendations(model_knn, movie_features_df, query_movie_title, n_recommendations)
print(recommended_movies)


                                 Movie Title  Distance
0                         Toy Story 2 (1999)     0.427
1                       Jurassic Park (1993)     0.434
2       Independence Day (a.k.a. ID4) (1996)     0.436
3  Star Wars: Episode IV - A New Hope (1977)     0.443
4                        Forrest Gump (1994)     0.453


# Preparing the User-Item Matrix #

In [42]:
user_movie_rating_matrix = ratings_df.pivot_table(index='userId', columns='movieId', values='rating').fillna(0)


In [43]:
user_movie_rating_matrix.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.000,0.000,4.000,0.000,0.000,4.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
4,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
5,4.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


Creating the Matrix for kNN

In [44]:
from scipy.sparse import csr_matrix

user_movie_rating_matrix_sparse = csr_matrix(user_movie_rating_matrix.values)


Fitting the kNN Model

In [45]:
from sklearn.neighbors import NearestNeighbors

# Initialize the model
model_knn_user = NearestNeighbors(metric='cosine', algorithm='brute')

# Fit the model
model_knn_user.fit(user_movie_rating_matrix_sparse)


NearestNeighbors(algorithm='brute', metric='cosine')

In [46]:
def user_based_recommendations(user_id, user_movie_rating_matrix, model_knn_user, n_neighbors=5):
     # Ensure the user exists in the matrix
    if user_id not in user_movie_rating_matrix.index:
        return "User ID not found."
    
    distances, indices = model_knn_user.kneighbors(user_movie_rating_matrix.loc[user_id,:].values.reshape(1, -1), n_neighbors=n_neighbors+1)
    
    similar_users_indices = indices.flatten()[1:]  # exclude the first one since it's the query user itself
    
    recommended_movies = []
    for similar_user_index in similar_users_indices:
        similar_user_id = user_movie_rating_matrix.index[similar_user_index]
        top_movies = user_movie_rating_matrix.loc[similar_user_id].sort_values(ascending=False).head(3).index
        recommended_movies.extend(top_movies)
    
    # Filter out movies the query user has already seen/rated, updated for newer pandas versions
    seen_movies = user_movie_rating_matrix.columns[user_movie_rating_matrix.loc[user_id] > 0].tolist()
    recommended_movies = [movie for movie in recommended_movies if movie not in seen_movies]
    
    # Return unique recommended movie IDs
    return list(set(recommended_movies))[:n_neighbors]


In [47]:
user_id = 1  # Example user ID
recommended_movie_ids = user_based_recommendations(user_id, user_movie_rating_matrix, model_knn_user)
print(recommended_movie_ids)


[6659, 1285, 904, 1387, 4941]
